In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

clf = pickle.load(open("classifier.pck", "rb"))
enc = pickle.load(open("encoder.pck", "rb"))

next_matchday_df = pickle.load(open("next_matchday_df.pck", "rb"))
next_matchday_df.drop("resultClass", axis=1, inplace=True)
next_matchday_df_prep = next_matchday_df.drop(["date", "teamHomeName", "teamAwayName"], axis=1)
predictions = clf.predict(next_matchday_df_prep)
next_matchday_df["prediction"] = enc.inverse_transform(predictions)
next_matchday_df[["date", "teamHomeName", "teamAwayName", "prediction"]]

,date,teamHomeName,teamAwayName,prediction
11409,2024-11-08 20:30:00,1. FC Union Berlin,SC Freiburg,1:1
11410,2024-11-09 15:30:00,Werder Bremen,Holstein Kiel,2:1
11411,2024-11-09 15:30:00,1. FSV Mainz 05,Borussia Dortmund,1:1
11412,2024-11-09 15:30:00,VfL Bochum,Bayer Leverkusen,1:1
11413,2024-11-09 15:30:00,FC St. Pauli,FC Bayern München,1:1
11414,2024-11-09 18:30:00,RB Leipzig,Borussia Mönchengladbach,1:1
11415,2024-11-10 15:30:00,FC Augsburg,TSG 1899 Hoffenheim,1:1
11416,2024-11-10 17:30:00,VfB Stuttgart,Eintracht Frankfurt,2:0
11417,2024-11-10 19:30:00,1. FC Heidenheim 1846,VfL Wolfsburg,1:1


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import dotenv
import os

dotenv.load_dotenv(".env")
os.getenv("EMAIL")

driver = webdriver.Chrome()
driver.get(os.getenv("LINK-TIPPABGABE"))

nav_button = driver.find_element(By.CLASS_NAME, "navtoggle")
nav_button.click()

email_input = driver.find_element(By.ID, "kennung")
email_input.send_keys(os.getenv("EMAIL"))
passwort_input = driver.find_element(By.ID, "passwort")
passwort_input.send_keys(os.getenv("PASSWORT"))

login_button = driver.find_element(By.NAME, "submitbutton")
login_button.click()

In [3]:
cookies_div = driver.find_element(By.CLASS_NAME, "qc-cmp2-summary-buttons")
for button in cookies_div.find_elements(By.TAG_NAME, "button"):
    if button.find_element(By.TAG_NAME, "span").get_attribute("innerHTML") == "ZUSTIMMEN":
        button.click()
        break

In [4]:
from difflib import get_close_matches

tippabgabe_tabelle = driver.find_element(By.ID, "tippabgabeSpiele")
for datarow in tippabgabe_tabelle.find_elements(By.CLASS_NAME, "datarow"):
    homeTeam = datarow.find_element(By.CLASS_NAME, "col1").get_attribute("innerHTML")
    if homeTeam != "":
        selectedTeam = get_close_matches(homeTeam, next_matchday_df["teamHomeName"])[0]
        prediction = next_matchday_df[next_matchday_df["teamHomeName"] == selectedTeam]["prediction"].iloc[0].split(":")
        inputs = datarow.find_elements(By.TAG_NAME, "input")
        inputs[1].clear()
        inputs[1].send_keys(prediction[0])
        inputs[2].clear()
        inputs[2].send_keys(prediction[1])

In [5]:
driver.find_element(By.NAME, "submitbutton").click()

In [6]:
driver.close()